In [1]:
import numpy as np
import pandas as pd
import json
import gzip
import ast
import matplotlib.pyplot as plt
import string

from urllib.request import urlopen
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from collections import defaultdict
from random import sample 
from scipy.sparse import lil_matrix
from nltk.stem.porter import *
from collections import Counter
import nltk
from nltk.corpus import stopwords 
from collections import defaultdict

In [11]:
def parseDataFromFile(fname):
    for l in open(fname):
        yield ast.literal_eval(l)

data = list(parseDataFromFile("train_Category.json"))
train_data = data[:165000]
val_data = data[165000:165000 + 10000]

wordCount = Counter()
totalWords = 0

punct = string.punctuation
stop_words = set(stopwords.words('english'))

for d in train_data:
    t = d['text']
    t = t.lower() # lowercase string


    t = t.translate(str.maketrans('', '', string.punctuation))
    t = ''.join(t) # convert back to string
    words = t.strip().split() # tokenizes
    for w in words:
        totalWords += 1
        wordCount[w] += 1

words = wordCount.most_common(17000)
words = [x[0] for x in words if x[0] not in stopwords.words('english')]
wordId = dict(zip(words, range(len(words))))

counts = [(wordCount[w], w) for w in wordCount]

In [16]:
def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['text'].lower() if not c in punct])
    for w in r.split():
        if w in words:

            feat[wordId[w]] += 1


    feat.append(1) #offset
    return feat

In [17]:
X_train = lil_matrix([feature(d) for d in train_data])
y_train = [x['genreID'] for x in train_data]

In [ ]:
X_val = lil_matrix([feature(d) for d in val_data])
y_val = [x['genreID'] for x in val_data]

In [ ]:
mod = linear_model.LogisticRegression(C=10000)
mod.fit(X_train, y_train)

pred = mod.predict(X_val)
accuracy_score(pred, y_val)

In [21]:
### kaggle stuff
test_data = list(parseDataFromFile("test_Category.json"))

X_test = [feature(d) for d in test_data]
X_test = lil_matrix(X_test)

pred_kaggle = mod.predict(X_test)

In [22]:
predictions = open("predictions_Category.txt", 'w')
predictions.write('userID-reviewID,prediction\n')

for i in range(len(test_data)):
    u,g = test_data[i]['userID'], test_data[i]['reviewID']

    predictions.write(u + '-' + g + "," + str(pred_kaggle[i]) + "\n")


predictions.close()